In [42]:
import lightgbm as lgbm
from scipy import sparse as ssp
from sklearn.model_selection import StratifiedKFold

import pandas as pd
#显示所有列
pd.set_option("display.max_columns",  None)
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [43]:
def Gini(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]
    print(y_true.shape)
    
    # 排序
    arr = np.array([y_true, y_pred]).transpose()
    print(arr.shape)
    true_order = arr[arr[:, 0].argsort()][::-1, 0] # 从小到大排列，输出对应的index
    pred_order = arr[arr[:, 1].argsort()][::-1, 0]
    
    # get Lorenz curves
    L_true = np.cumsum(true_order) * 1. / np.sum(true_order)
    L_pred = np.cumsum(pred_order) * 1. / np.sum(pred_order)
    L_ones = np.linspace(1 / n_samples, 1, n_samples) # 步长
    
    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)
    
    # normalize to true Gini coefficient
    return G_pred * 1. / G_true

In [68]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return "gini", Gini(labels, preds), True

In [45]:
path = "./input/"

train = pd.read_csv(path + "train.csv")
train_label = train["target"]
train_id = train["id"]
test = pd.read_csv(path + "test.csv")
test_id = test["id"]

In [46]:
train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.718070,10,1,-1,0,1,4,1,0,0,1,12,2,0.400000,0.883679,0.370810,3.605551,0.6,0.5,0.2,3,1,10,1,10,1,5,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316228,0.618817,0.388716,2.449490,0.3,0.1,0.3,2,1,9,5,8,1,7,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.000000,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316228,0.641586,0.347275,3.316625,0.5,0.7,0.1,2,2,9,1,8,2,7,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,7,1,0,0,1,11,1,1,3,1,104,1,0.374166,0.542949,0.294958,2.000000,0.6,0.9,0.1,2,4,7,1,8,4,2,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840759,11,1,-1,0,-1,14,1,1,2,1,82,3,0.316070,0.565832,0.365103,2.000000,0.4,0.6,0.0,2,2,6,3,10,2,12,3,1,1,3,0,0,0,1,1,0


In [47]:
train.describe()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,5.952120e+05,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,7.438036e+05,0.036448,1.900378,1.358943,4.423318,0.416794,0.405188,0.393742,0.257033,0.163921,0.185304,0.000373,0.001692,0.009439,0.000948,0.012451,7.299922,0.660823,0.121081,0.153446,0.610991,0.439184,0.551102,8.295933,0.829931,-0.504899,0.725192,-0.157732,6.555340,0.910027,0.832080,1.328890,0.992136,62.215674,2.346072,0.379945,0.813265,0.276256,3.065899,0.449756,0.449589,0.449849,2.372081,1.885886,7.689445,3.005823,9.225904,2.339034,8.433590,5.441382,1.441918,2.872288,7.539026,0.122427,0.627840,0.554182,0.287182,0.349024,0.153318
std,4.293678e+05,0.187401,1.983789,0.664594,2.699902,0.493311,1.350642,0.488579,0.436998,0.370205,0.388544,0.019309,0.041097,0.096693,0.030768,0.127545,3.546042,0.473430,0.326222,0.360417,0.287643,0.404264,0.793506,2.508270,0.375716,0.788654,2.153463,0.844417,5.501445,0.347106,0.373796,0.978747,0.091619,33.012455,0.832548,0.058327,0.224588,0.357154,0.731366,0.287198,0.286893,0.287153,1.117219,1.134927,1.334312,1.414564,1.459672,1.246949,2.904597,2.332871,1.202963,1.694887,2.746652,0.327779,0.483381,0.497056,0.452447,0.476662,0.360295
min,7.000000e+00,0.000000,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,1.000000,-1.000000,-1.000000,0.250619,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.719915e+05,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.400000,0.200000,0.525000,7.000000,1.000000,-1.000000,0.000000,-1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,32.000000,2.000000,0.316228,0.670867,0.333167,2.828427,0.200000,0.200000,0.200000,2.000000,1.000000,7.000000,2.000000,8.000000,1.000000,6.000000,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.435475e+05,0.000000,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,1.000000,0.000000,0.000000,0.700000,0.300000,0.720677,7.000000,1.000000,-1.000000,0.000000,0.000000,7.0000

In [48]:
NFOLDS = 5
kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=218)

In [49]:
y = train["target"].values
print(y.shape)

(595212,)


In [50]:
drop_feature = ["id", "target"]

In [51]:
len(train.columns)

59

In [52]:
train.columns

Index(['id', 'target', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03',
       'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin',
       'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin',
       'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15',
       'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01',
       'ps_reg_02', 'ps_reg_03', 'ps_car_01_cat', 'ps_car_02_cat',
       'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat',
       'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat',
       'ps_car_11_cat', 'ps_car_11', 'ps_car_12', 'ps_car_13', 'ps_car_14',
       'ps_car_15', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_calc_04',
       'ps_calc_05', 'ps_calc_06', 'ps_calc_07', 'ps_calc_08', 'ps_calc_09',
       'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 'ps_calc_14',
       'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin',
       'ps_calc_19_bin', 'ps_calc_20_bin'],


In [53]:
X = train.drop(drop_feature, axis=1)
feature_names = X.columns.tolist() # 特征名称

# bin是binary features， cat是categorical features， calc是group交叉特征
cat_features = [c for c in feature_names if ("cat" in c and "count" not in c)] # 类别特征
num_features = [c for c in feature_names if ("cat" not in c and "calc" not in c)] # 数值特征
print(len(cat_features), len(num_features))

14 23


In [54]:
# Values of -1 indicate that the feature was missing from the observation 获取missing的-1特征值
train["missing"] = (train == -1).sum(axis=1).astype(float) # 统计每行特征值为-1的数量
test["missing"] = (test == -1).sum(axis=1).astype(float)
num_features.append("missing")
train["missing"]

0         1.0
1         2.0
2         3.0
3         0.0
4         2.0
         ... 
595207    1.0
595208    3.0
595209    2.0
595210    2.0
595211    3.0
Name: missing, Length: 595212, dtype: float64

In [56]:
ind_features = [c for c in feature_names if "ind" in c]
ind_features

['ps_ind_01',
 'ps_ind_02_cat',
 'ps_ind_03',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_ind_06_bin',
 'ps_ind_07_bin',
 'ps_ind_08_bin',
 'ps_ind_09_bin',
 'ps_ind_10_bin',
 'ps_ind_11_bin',
 'ps_ind_12_bin',
 'ps_ind_13_bin',
 'ps_ind_14',
 'ps_ind_15',
 'ps_ind_16_bin',
 'ps_ind_17_bin',
 'ps_ind_18_bin']

In [57]:
# LabelENcoder
for c in cat_features:
    le = LabelEncoder()
    le.fit(train[c])
    train[c] = le.transform(train[c])
    test[c] = le.transform(test[c])
    
# OneHotEncoder
enc = OneHotEncoder()
enc.fit(train[cat_features])
X_cat = enc.transform(train[cat_features])
X_t_cat = enc.transform(test[cat_features])

# ind特征
ind_features = [c for c in feature_names if "ind" in c]
count = 0
for c in ind_features:
    if count == 0:
        train["new_ind"] = train[c].astype(str) + "_"
        test["new_ind"] = test[c].astype(str) + "_"
        count += 1
    else:
        train["new_ind"] += train[c].astype(str) + "_"
        test["new_ind"] += test[c].astype(str) + "_"
train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,missing,new_ind
0,7,0,2,2,5,2,1,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.718070,11,2,0,0,2,4,2,0,1,1,11,2,0.400000,0.883679,0.370810,3.605551,0.6,0.5,0.2,3,1,10,1,10,1,5,9,1,5,8,0,1,1,0,0,1,1.0,2_2_5_2_1_0_1_0_0_0_0_0_0_0_11_0_1_0_
1,9,0,1,1,7,1,1,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,12,2,0,0,0,11,2,1,3,1,18,3,0.316228,0.618817,0.388716,2.449490,0.3,0.1,0.3,2,1,9,5,8,1,7,3,1,1,9,0,1,1,0,1,0,2.0,1_1_7_1_1_0_0_1_0_0_0_0_0_0_3_0_0_1_
2,13,0,5,4,9,2,1,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.000000,8,2,0,0,0,14,2,1,3,1,59,1,0.316228,0.641586,0.347275,3.316625,0.5,0.7,0.1,2,2,9,1,8,2,7,4,2,7,7,0,1,1,0,1,0,3.0,5_4_9_2_1_0_0_1_0_0_0_0_0_0_12_1_0_0_
3,16,0,0,1,2,1,1,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,8,2,1,0,2,11,2,1,4,1,103,1,0.374166,0.542949,0.294958,2.000000,0.6,0.9,0.1,2,4,7,1,8,4,2,2,2,4,9,0,0,0,0,0,0,0.0,0_1_2_1_1_1_0_0_0_0_0_0_0_0_8_1_0_0_
4,17,0,0,2,0,2,1,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840759,12,2,0,0,0,14,2,1,3,1,81,3,0.316070,0.565832,0.365103,2.000000,0.4,0.6,0.0,2,2,6,3,10,2,12,3,1,1,3,0,0,0,1,1,0,2.0,0_2_0_2_1_1_0_0_0_0_0_0_0_0_9_1_0_0_


In [61]:
cat_count_features = []
for c in cat_features + ["new_ind"]:
    d = pd.concat([train[c],test[c]]).value_counts().to_dict()
    train["%s_count" % c] = train[c].apply(lambda x:d.get(x, 0)) # 计算(train&test)下每个特征值的数量 
    test["%s_count" % c] = test[c].apply(lambda x:d.get(x, 0))
    cat_count_features.append("%s_count" % c)

In [66]:
train[num_features + cat_count_features].values.shape, test[num_features + cat_count_features].values.shape

((595212, 39), (892816, 39))

In [64]:
X_cat.shape, X_cat.shape

((595212, 184), (595212, 184))

In [63]:
train_list = [train[num_features + cat_count_features].values, X_cat,] # 与onehot处理后的特征连接
test_list = [test[num_features + cat_count_features].values, X_t_cat,]

In [67]:
X = ssp.hstack(train_list).tocsr() # 行连接，转换为稀疏矩阵
X_test = ssp.hstack(test_list).tocsr()

In [69]:
test.head()

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,missing,new_ind,ps_ind_02_cat_count,ps_ind_04_cat_count,ps_ind_05_cat_count,ps_car_01_cat_count,ps_car_02_cat_count,ps_car_03_cat_count,ps_car_04_cat_count,ps_car_05_cat_count,ps_car_06_cat_count,ps_car_07_cat_count,ps_car_08_cat_count,ps_car_09_cat_count,ps_car_10_cat_count,ps_car_11_cat_count,new_ind_count
0,0,0,1,8,2,1,0,1,0,0,0,0,0,0,0,12,1,0,0,0.5,0.3,0.610328,8,2,0,0,0,1,2,1,3,1,64,1,0.316228,0.669556,0.352136,3.464102,0.1,0.8,0.6,1,1,6,3,6,2,9,1,1,1,12,0,1,1,0,0,1,2.0,0_1_8_2_1_0_1_0_0_0_0_0_0_0_12_1_0_0_,1079327,620936,1319412,449617,1234979,1028142,1241334,666910,295574,1383070,1238365,883326,1475460,27800,164
1,1,4,2,5,2,1,0,0,0,1,0,0,0,0,0,5,1,0,0,0.9,0.5,0.771362,5,2,0,0,1,11,2,1,1,1,102,1,0.316228,0.606320,0.358329,2.828427,0.4,0.5,0.4,3,3,8,4,10,2,7,2,0,3,10,0,0,1,1,0,1,1.0,4_2_5_2_1_0_0_0_1_0_0_0_0_0_5_1_0_0_,309747,620936,1319412,65720,1234979,1028142,1241334,389558,329890,1383070,1238365,486510,1475460,61062,8
2,2,5,1,3,1,1,0,0,0,1,0,0,0,0,0,10,0,0,0,0.4,0.0,0.916174,12,2,0,0,0,14,2,1,3,1,28,3,0.400000,0.896239,0.398497,3.316625,0.6,0.6,0.6,2,3,7,4,6,3,12,4,0,2,4,0,0,0,0,0,0,2.0,5_1_3_1_1_0_0_0_1_0_0_0_0_0_10_0_0_0_,1079327,866864,1319412,518725,1234979,1028142,1241334,666910,147714,1383070,1238365,883326,1475460,10957,57
3,3,0,1,6,1,1,1,0,0,0,0,0,0,0,0,4,1,0,0,0.1,0.2,-1.000000,8,2,0,0,0,1,2,1,3,1,39,2,0.374166,0.652110,0.381445,2.449490,0.1,0.5,0.5,2,1,7,3,12,1,13,5,1,0,5,1,0,1,0,0,0,3.0,0_1_6_1_1_1_0_0_0_0_0_0_0_0_4_1_0_0_,1079327,866864,1319412,449617,1234979,1028142,1241334,666910,295574,1383070,1238365,883326,1475460,12391,477
4,4,5,1,7,1,1,0,0,0,1,0,0,0,0,0,4,1,0,0,0.9,0.4,0.817771,12,2,0,0,0,11,2,1,3,1,100,3,0.374166,0.812914,0.385097,3.316625,0.9,0.6,0.8,3,4,7,1,10,4,12,4,0,0,4,0,1,1,0,0,1,2.0,5_1_7_1_1_0_0_0_1_0_0_0_0_0_4_1_0_0_,1079327,866864,1319412,518725,1234979,1028142,1241334,666910,329890,1383070,1238365,883326,1475460,18416,45


In [79]:
# lgbm参数
learning_rate = 0.1
num_leaves = 15
min_data_in_leaf = 2000
feature_fraction = 0.6
num_boost_round = 10000
params = {"objective": "binary",
          "boosting_type": "gbdt",
          "learning_rate": learning_rate,
          "num_leaves": num_leaves,
           "max_bin": 256,
          "feature_fraction": feature_fraction,
          "verbosity": 0,
          "drop_rate": 0.1,
          "is_unbalance": False,
          "max_drop": 50,
          "min_child_samples": 10,
          "min_child_weight": 150,
          "min_split_gain": 0,
          "subsample": 0.9
          }

cv_only = True
save_cv = True
full_train = False

x_score = []
ct = 0
final_cv_train = np.zeros(len(train_label))
final_cv_pred = np.zeros(len(test_id))

from tqdm import tqdm_notebook

for s in tqdm_notebook(range(16)):
    cv_train = np.zeros(len(train_label))
    cv_pred = np.zeros(len(test_id))
    
    params["seed"] = s
    
    # kfold
    if cv_only:
        kf = kfold.split(X, train_label)
        
        best_trees = []
        fold_scores = []
        
        for i, (train_fold, validate) in enumerate(kf):
            X_train, X_validate, label_train, label_validate = X[train_fold, :], X[validate, :], train_label[train_fold], train_label[validate]
            # LGBM
            dtrain = lgbm.Dataset(X_train, label_train)
            dvalid = lgbm.Dataset(X_validate, label_validate, reference=dtrain)
            bst = lgbm.train(params, dtrain, num_boost_round, valid_sets=dvalid, feval=evalerror,
                            verbose_eval=100,early_stopping_rounds=100)
            best_trees.append(bst.best_iteration)
            cv_pred += bst.predict(X_test, num_iteration=bst.best_iteration)
            cv_train[validate] += bst.predict(X_validate)
            
            score = Gini(label_validate, cv_train[validate])
            print(score)
            fold_scores.append(score)
            
        cv_pred /= NFOLDS # 除以划分次数
        final_cv_train += cv_train
        final_cv_pred += cv_pred
        
        print("cv score:")
        print(Gini(train_label, cv_train))
        print("current score:", Gini(train_label, final_cv_train / (ct + 1.)))
        print(fold_scores)
        print(best_trees, np.mean(best_trees))
        ct += 1
        
        x_score.append(Gini(train_label, cv_train))
        
print(x_score)
# pd.DataFrame({'id': test_id, 'target': final_cv_pred / 16.}).to_csv('./model/lgbm3_pred_avg.csv', index=False)
# pd.DataFrame({'id': train_id, 'target': final_cv_train / 16.}).to_csv('./model/lgbm3_cv_avg.csv', index=False)

F:\anaconda3\lib\site-packages\ipykernel_launcher.py:34: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


F:\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
F:\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.110932 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
(119043,)
(119043, 2)
Training until validation scores don't improve for 100 rounds
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(1190

(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)


(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
[100]	valid_0's binary_logloss: 0.152078	valid_0's gini: 0.275372
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)


...

(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
[100]	valid_0's binary_logloss: 0.152047	valid_0's gini: 0.27558
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(119043, 2)
(119043,)
(

(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
[100]	valid_0's binary_logloss: 0.152038	valid_0's gini: 0.278033
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)


(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
[100]	valid_0's binary_logloss: 0.151786	valid_0's gini: 0.284533
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)


(119042,)
(119042, 2)
(119042,)
(119042, 2)
(119042,)
(119042, 2)
Early stopping, best iteration is:
[124]	valid_0's binary_logloss: 0.151419	valid_0's gini: 0.290163
(119042,)
(119042, 2)
0.29016263144257903
cv score:
(595212,)
(595212, 2)
0.2860899794511859
(595212,)
(595212, 2)
current score: 0.2888681171481047
[0.29419368095026177, 0.27993981989045236, 0.28215180399626705, 0.284796817172348, 0.29016263144257903]
[169, 224, 243, 113, 124] 174.6
(595212,)
(595212, 2)

[0.28581087287246515, 0.28612703231439995, 0.28523677934240615, 0.2867214013659066, 0.28570773588756787, 0.2862086579536886, 0.2862973628978743, 0.28701986704651966, 0.28641085376805125, 0.28608409208647545, 0.287066334227867, 0.2860178438419425, 0.2857536785509344, 0.28594513163263857, 0.2858514964123292, 0.2860899794511859]


FileNotFoundError: [Errno 2] No such file or directory: '../model/lgbm3_pred_avg.csv'